In [55]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error as mse, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from chaining import Chain
import os

In [56]:
X, y = load_iris(return_X_y=True)
y = np.vstack((X[:,-1], y)).T
X = X[:, :-1]

In [57]:
# Create and fit the chain
chain = Chain(
    model_reg=RandomForestRegressor(random_state=42),
    model_clf=RandomForestClassifier(random_state=42),
    propagate="pred",
)
chain.fit(X, y, target_types=["reg","clf"])

# Make predictions
y_pred = chain.predict(X)

# Calculate and print MSE
scores = [ 
    mse(y[:,0], y_pred.iloc[:,0]),
    mse(y[:,1], y_pred.iloc[:,1]),
]
print(f"MSE = {scores[0]:.2f}, {scores[1]:.2f}")

MSE = 0.01, 0.00


---

Our model

In [58]:
# Function for assigning the folds
def missingness_stratified_cv(df, N_FOLDS=5):
    
    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [59]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [60]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

In [61]:
# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)

In [62]:
data = pd.read_csv(path)
data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,DS,DS_L,DS_R,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,RAND36_M-before,RAND36_M-after,RAND36_P-before,RAND36_P-after,SF12_M-before,SF12_M-after,SF12_P-before,SF12_P-after
0,MSOAC/0649,NaN,F,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.35,10.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/2224,38.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,0,1,0,0,NaN,NaN,27.80,26.55,NaN,NaN,NaN,NaN,55.0,56.0,NaN,38.0,42.0,8.90,8.85,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,30.0,30.0,1.25,1.25,NaN,0.380952,0.388889,3.00,3.0,NaN,0.000000,NaN,0.000000,0.2,NaN,0.2,0.000000,NaN,0.000000,0.5,NaN,0.500000,0.0,NaN,0.0,0.000000,NaN,0.0,0.0,NaN,0.0,0.148148,NaN,0.148148,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,0.8,0.7,0.5,0.5
2,MSOAC/0576,50.0,F,WHITE,NaN,NaN,NaN,NaN,Y,PPMS,0,1,0,1,NaN,NaN,17.75,18.65,17.90,51.0,56.5,56.5,59.0,59.0,59.5,NaN,NaN,4.55,4.70,5.15,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,4.0,4.25,0.333333,0.250000,0.333333,0.0,0.0,0.1,0.250000,0.166667,0.333333,0.5,0.500000,0.500000,0.0,0.2,0.0,0.083333,0.083333,0.0,0.4,0.4,0.3,0.240741,0.277778,0.259259,0.166667,0.125000,0.166667,0.628571,0.671429,0.594937,0.531646,NaN,NaN,NaN,NaN
3,MSOAC/4961,44.0,F,WHITE,NaN,NaN,NaN,NaN,Y,PPMS,0,1,1,1,NaN,NaN,19.75,19.45,20.60,38.5,52.0,54.0,55.5,59.0,58.0,NaN,NaN,4.00,4.90,5.60,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.50,3.5,3.50,0.333333,0.250000,0.333333,0.0,0.1,0.0,0.166667,0.250000,0.250000,0.5,0.500000,0.416667,0.0,0.2,0.0,0.000000,0.000000,0.0,0.2,0.3,0.3,0.185185,0.277778,0.222222,0.083333,0.125000,0.125000,0.614286,0.614286,0.582278,0.645570,NaN,NaN,NaN,NaN
4,MSOAC/5990,52.0,F,WHITE,NaN,NaN,NaN,NaN,Y,PPMS,1,1,0,1,NaN,NaN,21.45,21.35,22.75,22.0,23.5,22.0,35.0,41.5,44.0,NaN,NaN,11.40,16.80,30.20,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,6.0,6.50,0.166667,0.333333,0.166667,0.2,0.2,0.2,0.166667,0.166667,0.166667,0.5,0.666667,0.500000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.4,0.0,0.222222,0.333333,0.185185,0.083333,0.166667,0.083333,0.600000,0.742857,0.506329,0.531646,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/2501,46.0,F,WHITE,NaN,NaN,NaN,12.0,NaN,SPMS,0,0,0,0,NaN,NaN,29.40,25.65,NaN,14.0,12.0,NaN,17.5,16.0,NaN,NaN,NaN,9.30,8.10,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.642857,0.614286,0.645570,0.588608,NaN,NaN,NaN,NaN
2461,MSOAC/8672,43.0,F,NaN,NaN,MODERATE,NaN,1.0,NaN,RRMS,

In [63]:
data['NRELAP'] = data['NRELAP'].fillna(0)

def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'

# Assuming your DataFrame is df and the column to bin is 'column_name'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

In [64]:
variables = ['EDSS-2y', 'T25FW-2y', 'RAND36_P-after', 'NRELAP']# , 'CESEV' ('SMSTDY' gave a score of -0.03)

# Extract targets
targets = data[variables]

# Extract features by dropping the target columns
features = data.drop(variables, axis=1)

In [65]:
columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'RAND36_M-before', 'RAND36_P-before', 'SF12_M-before', 
                    'SF12_P-before']
# still need to change in OE dataframe the SLEC and SES so name is consistent with the others

features = features[columns_to_keep]
features

,AGE,SEX,RACE,CONTINENT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,RAND36_M-before,RAND36_P-before,SF12_M-before,SF12_P-before
0,NaN,F,WHITE,NORTH AMERICA,SPMS,0,1,0,1,NaN,NaN,NaN,NaN,11.35,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,F,WHITE,EUROPE,RRMS,0,1,0,0,27.80,NaN,55.0,38.0,8.90,30.0,1.25,0.380952,3.00,0.148148,0.000000,NaN,NaN,0.8,0.5
2,50.0,F,WHITE,NaN,PPMS,0,1,0,1,17.75,51.0,59.0,NaN,4.55,NaN,NaN,NaN,3.75,0.259259,0.166667,0.628571,0.594937,NaN,NaN
3,44.0,F,WHITE,NaN,PPMS,0,1,1,1,19.75,38.5,55.5,NaN,4.00,NaN,NaN,NaN,3.50,0.222222,0.125000,0.614286,0.582278,NaN,NaN
4,52.0,F,WHITE,NaN,PPMS,1,1,0,1,21.45,22.0,35.0,NaN,11.40,NaN,NaN,NaN,6.00,0.185185,0.083333,0.600000,0.506329,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.0,F,WHITE,NaN,SPMS,0,0,0,0,29.40,14.0,17.5,NaN,9.30,NaN,NaN,NaN,6.00,NaN,NaN,0.642857,0.645570,NaN,NaN
2461,43.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,30.0,M,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,42.0,M,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)

In [67]:
# Create a new column 'RAND36_M-before' and fill it with values from 'SF12_M-before'
features['M_R36-SF12-before'] = features['RAND36_M-before'].fillna(features['SF12_M-before'])

# Create a new column 'R36-SF12' with a binary indicator
features['M_R36-SF12'] = features.apply(lambda row: 1 if pd.notna(row['RAND36_M-before']) else (0 if pd.notna(row['SF12_M-before']) else np.nan), axis=1)

# Drop the original columns if needed
features = features.drop(['SF12_M-before'], axis=1)

# Create a new column 'RAND36_M-before' and fill it with values from 'SF12_M-before'
features['P_R36-SF12-before'] = features['RAND36_P-before'].fillna(features['SF12_P-before'])

# Create a new column 'R36-SF12' with a binary indicator
features['P_R36-SF12'] = features.apply(lambda row: 1 if pd.notna(row['RAND36_P-before']) else (0 if pd.notna(row['SF12_P-before']) else np.nan), axis=1)

# Drop the original columns if needed
features = features.drop(['SF12_P-before'], axis=1)
features

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,RAND36_M-before,RAND36_P-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS,M_R36-SF12-before,M_R36-SF12,P_R36-SF12-before,P_R36-SF12
0,NaN,0,1,0,1,NaN,NaN,NaN,NaN,11.35,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,1,0,0,1,0,0,0,1,0,0,0,0,1,NaN,NaN,NaN,NaN
1,38.0,0,1,0,0,27.80,NaN,55.0,38.0,8.90,30.0,1.25,0.380952,3.00,0.148148,0.000000,NaN,NaN,1,0,0,1,0,0,1,0,0,0,0,1,0,0.800000,0.0,0.500000,0.0
2,50.0,0,1,0,1,17.75,51.0,59.0,NaN,4.55,NaN,NaN,NaN,3.75,0.259259,0.166667,0.628571,0.594937,1,0,0,1,0,0,0,0,0,0,1,0,0,0.628571,1.0,0.594937,1.0
3,44.0,0,1,1,1,19.75,38.5,55.5,NaN,4.00,NaN,NaN,NaN,3.50,0.222222,0.125000,0.614286,0.582278,1,0,0,1,0,0,0,0,0,0,1,0,0,0.614286,1.0,0.582278,1.0
4,52.0,1,1,0,1,21.45,22.0,35.0,NaN,11.40,NaN,NaN,NaN,6.00,0.185185,0.083333,0.600000,0.506329,1,0,0,1,0,0,0,0,0,0,1,0,0,0.600000,1.0,0.506329,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.0,0,0,0,0,29.40,14.0,17.5,NaN,9.30,NaN,NaN,NaN,6.00,NaN,NaN,0.642857,0.645570,1,0,0,1,0,0,0,0,0,0,0,0,1,0.642857,1.0,0.645570,1.0
2461,43.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
2462,30.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN
2463,42.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN


In [68]:
targets.dtypes

EDSS-2y           float64
T25FW-2y          float64
RAND36_P-after    float64
NRELAP             object
dtype: object

In [69]:
# Set random state for reproducibility
random_state = 42

In [70]:
# Generate CV folds
cv=missingness_stratified_cv(features, random_state=random_state)
cv = cv.to_frame(name="CV Fold")
features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

features_cv.head()

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,RAND36_M-before,RAND36_P-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS,M_R36-SF12-before,M_R36-SF12,P_R36-SF12-before,P_R36-SF12,CV Fold
0,NaN,0,1,0,1,NaN,NaN,NaN,NaN,11.35,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,1,0,0,1,0,0,0,1,0,0,0,0,1,NaN,NaN,NaN,NaN,1.0
1,38.0,0,1,0,0,27.80,NaN,55.0,38.0,8.90,30.0,1.25,0.380952,3.00,0.148148,0.000000,NaN,NaN,1,0,0,1,0,0,1,0,0,0,0,1,0,0.800000,0.0,0.500000,0.0,3.0
2,50.0,0,1,0,1,17.75,51.0,59.0,NaN,4.55,NaN,NaN,NaN,3.75,0.259259,0.166667,0.628571,0.594937,1,0,0,1,0,0,0,0,0,0,1,0,0,0.628571,1.0,0.594937,1.0,2.0
3,44.0,0,1,1,1,19.75,38.5,55.5,NaN,4.00,NaN,NaN,NaN,3.50,0.222222,0.125000,0.614286,0.582278,1,0,0,1,0,0,0,0,0,0,1,0,0,0.614286,1.0,0.582278,1.0,0.0
4,52.0,1,1,0,1,21.45,22.0,35.0,NaN,11.40,NaN,NaN,NaN,6.00,0.185185,0.083333,0.600000,0.506329,1,0,0,1,0,0,0,0,0,0,1,0,0,0.600000,1.0,0.506329,1.0,4.0


In [71]:
y_pred_list = []
y_test_list = []

for i in range(0, 5):
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred",
    )
    chain.fit(Xi_train, yi_train, target_types=["reg","reg","reg","clf"]) #,"clf" 
    y_pred = chain.predict(Xi_test)
    y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
    print("Done")

Done
Done
Done
Done
Done


In [72]:
y_pred_list[0]

,EDSS-2y,T25FW-2y,RAND36_P-after,NRELAP
3,3.5675,4.3105,0.605299,0.0
5,3.9625,5.2715,0.635245,1.0
7,3.9300,5.7910,0.589393,0.0
9,6.1800,24.2510,0.551461,0.0
11,6.6125,41.1080,0.501266,0.0
...,...,...,...,...
2446,6.5075,69.9550,0.483222,0.0
2450,3.9275,5.3265,0.637756,0.0
2452,6.2650,47.2820,0.577378,1.0
2454,6.5300,20.7965,0.496325,0.0


In [73]:
y_test_list[0]

,EDSS-2y,T25FW-2y,RAND36_P-after,NRELAP
3,3.5,4.90,0.645570,0.0
5,NaN,NaN,NaN,1.0
7,4.0,6.15,0.594937,1.0
9,6.5,23.45,0.481013,0.0
11,6.5,35.90,0.531646,0.0
...,...,...,...,...
2446,7.0,42.45,0.455696,1.0
2450,NaN,NaN,NaN,1.0
2452,6.5,36.30,NaN,0.0
2454,6.5,12.65,0.512658,1.0


In [74]:
#chain = Chain(
#        model_reg=RandomForestRegressor(random_state=42),
#        model_clf=RandomForestClassifier(random_state=42),
#        propagate="pred",
#    )
#print("Done 1")   
#chain.fit(features, targets, target_types=["reg","reg","reg","clf"]) #,"clf"
#print("Done 2")
#y_pred = chain.predict(features)
#print("Done 3")

In [75]:
len(y_test_list)

5

In [76]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

1st index: fold, 2nd index: outcome

In [77]:
y_test_cv[0][0]

3       3.5
7       4.0
9       6.5
11      6.5
20      1.5
       ... 
2440    6.0
2446    7.0
2452    6.5
2454    6.5
2456    2.5
Name: EDSS-2y, Length: 338, dtype: float64

In [78]:
y_pred_cv[0][3]

3       0.0
5       1.0
7       0.0
9       0.0
11      0.0
       ... 
2446    0.0
2450    0.0
2452    1.0
2454    0.0
2456    0.0
Name: NRELAP, Length: 500, dtype: object

In [79]:
# Remove corresponding rows where y_test is missing
#missing_rows_mask = targets.iloc[:, 0].isna()
#y_test1 = targets.iloc[:, 0][~missing_rows_mask]
#y_pred1 = y_pred.iloc[:, 0][~missing_rows_mask]

#missing_rows_mask = targets.iloc[:, 1].isna()
#y_test2 = targets.iloc[:, 1][~missing_rows_mask]
#y_pred2 = y_pred.iloc[:, 1][~missing_rows_mask]

#missing_rows_mask = targets.iloc[:, 2].isna()
#y_test3 = targets.iloc[:, 2][~missing_rows_mask]
#y_pred3 = y_pred.iloc[:, 2][~missing_rows_mask]

#missing_rows_mask = targets.iloc[:, 3].isna()
#y_test4 = targets.iloc[:, 3][~missing_rows_mask]
#y_pred4 = y_pred.iloc[:, 3][~missing_rows_mask]

#missing_rows_mask = y_test.iloc[:, 4].isna()
#y_test5 = y_test.iloc[:, 4][~missing_rows_mask]
#y_pred5 = y_pred.iloc[:, 4][~missing_rows_mask]

In [80]:
# Initialize a list to store scores
scores = []
scores_with_std = []

# Iterate over each variable in the folds
for variable_index in range(len(y_test_cv[0])):
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variable_index]
        y_pred = y_pred_cv[fold_index][variable_index]
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation
print("Scores = ", end="")
for avg_score, std_score in scores_with_std:
    print(f"{avg_score:.2f} (± {std_score:.2f})", end=", ")
print("\b\b")  # Remove the trailing comma and space

Scores = 0.88 (± 0.01), 0.72 (± 0.11), 0.62 (± 0.03), 0.64 (± 0.02)


In [81]:
#scores = [ 
#    r2_score(y_test1, y_pred1), 
#    r2_score(y_test2, y_pred2), 
#    r2_score(y_test3, y_pred3),
#    accuracy_score(y_test4, y_pred4),
#    ]
#print(f"Scores = {scores[0]:.2f}, {scores[1]:.2f}, {scores[2]:.2f}, {scores[3]:.2f}") 

*Questions/Notes*: 
- using pred="true" gives always score 1 (weird)
- when we have "clf" then the chain gives problems when trying to predict the next targets (num or cat)
- this is wrong - i think we need to include cv / train-test split in the chaining.py file after obtaining the fully annotated dataset

---